- 출처 : https://tutorials.pytorch.kr/intermediate/char_rnn_classification_tutorial.html

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

In [2]:
def findFiles(path) : return glob.glob(path)

# f_path = '/content/drive/My Drive/korean_embedding/raw_data/data/data/names/*.txt'

In [ ]:
import unicodedata
import string

all_letters = string.ascii_letters + ".,;'"
n_letters = len(all_letters)

# 유니코드 문자열을 ASCII 로 변환
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
  )

print(unicodeToAscii('Ślusàrski'))

# 각 언어의 이름 목록인 category_lines 사전생성
category_lines = {}
all_categories = []

# 파일을 읽고 줄 단위로 분리
def readLines(filename):
    lines = open(filename, encoding = 'utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles(f_path):
    category = os.path.splitext(os.path.basename(filename))[0]
    #print(category)
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)


## 이름을 tensor 로 변경
- 활용하기 위해 텐서로 전환 : 1 x n_letters 인 one-hot vector 사용

In [ ]:
import torch

# all_letters 로 문자의 주소 찾기 : 'a' = 0
def letterToIndex(letter):
    return all_letters.find(letter)
  # all_letters.find('z') = 25 가 나온다


# 검증을 위해 한개의 문자를  1 x n_letters 텐서로 변환
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    # tensor의 첫행 letter 열에 1을 집어넣음
    return tensor

# 한 줄 (이름)을 <line_lenth x 1 x n_letters>, 또는 원핫 벡터 array 로 변경
def lineToTensor(line): # 모든 단어에 반복하며 원핫인코딩
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))
print(lineToTensor('Jones').size())

## 네트워크 생성
- autograd 전에 torch에서 RNN 생성은 여러 시간 단계 걸쳐서 계층의 매개변수를 복제하는 작업을 포함한다
- 계층은 은닉 상태, gradient 를 가지며, 이제 이것들은 그래프 자체에서 완전히 처리된다
- 직접 RNN 을 사용해보고자 한다

In [ ]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [ ]:
# 한 단계를 실행하려면 입력과 이전의 은닉상태 (처음에는 0으로 초기화) 를 전달해야 한다
# 각 언어의 확률(= output) 과 다음 은닉 상태(유지) 를 받음

input = letterToTensor('A')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

## 학습
### 학습준비
- 각 카테고리의 우도인, 네트워크 출력을 해석해야 함
  - 각 카테고리 우도로, 확률이 높은 이름과 카테고리 번호를 반환
- 가장 큰 값의 주소를 알기 위해 Tensor.topk 를 사용할 수 있음


In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1) # 텐서의 가장 큰 값 및 주소
    category_i = top_i[0].item() # 텐서에서 정수 값으로 변경
    return all_categories[category_i],  category_i

a, b = categoryFromOutput(output)

print(a)

print(b)

In [ ]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)] # 랜덤하게 행렬 내 값들을 가져온다

def randomTrainingExample():
    category = randomChoice(all_categories) 
    #print(category) # all_categories 에서 카테고리 1개 가져온다
    line = randomChoice(category_lines[category]) 
    #print(line) # 이를 category_lines 내에서 1줄씩 그 카테고리에 포함된 단어를 불러온다
    category_tensor = torch.tensor([all_categories.index(category)], dtype = torch.long)
    #print(category_tensor) # 카테고리 id
    line_tensor = lineToTensor(line)
    #print(line_tensor) # line(= 단어) 도 원핫인코딩된 텐서로 변형
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line = ', line)
    print(line_tensor.shape)

## 네트워크 학습 
- 네트워크 학습하는데 필요한 예시를 보여주고 추정
- RNN 마지막 계층이 nn.LogSoftmax 이므로 손실함수로 nn.NLLoss 사용

In [ ]:
criterion = nn.NLLLoss()

### 학습루프
1. 입력, 타겟 tensor 생성
2. 0으로 초기된 hidden layer 생성
3. 각 문자를 읽어들이기
  - 다음 문자를 위한 hidden 상태 유지 (= next_hidden)
4. 타겟과 최종 output 출력 비교
5. 역전파
6. 출력과 손실 반환

In [ ]:
learning_rate = 0.005 
# 너무 높게 설정하면 발산할 수 이쏙, 너무 낮으면 학습안됨

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden() # 0으로 된 hidden layer 생성
    rnn.zero_grad() # 초기 grad 0으로 만들기
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        # 한줄씩 ( [0, 0, 0, ...., 1, 0, ..., 0] 읽어들임
        # i 1개씩 rnn 을 거치면서 생성된 hidden 값을 들고 있다가 그 다음줄을 붙여준다
        # torch.cat()
        #print(hidden.shapㅔㅛ색e)

    loss = criterion(output, category_tensor)
    loss.backward()

    # 매개변수의 경사도에 학습률을 곱한 뒤 그 매개변수 값에 더한다
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha = -learning_rate)

    return output, loss.item()


#### 예시 데이터를 사용하여 실행예정, train 함수가 output, loss.item() 을 반환

In [ ]:
import time
import math

n_iters = 1000000
print_every = 5000
plot_every = 1000

# 도식화를 위한 손실 추적 
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    # 데이터는 1개를 여기서 가져온다
    
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss 

    # iter 숫자, 손실, 이름, 추측 화면 출력
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # 현재 평균 손실을 전체 손실 리스트에 추가
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0